In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import os
import sys
import csv
import math
from operator import itemgetter
import time

from sklearn.svm import SVC, SVR
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.externals import joblib
from sklearn.feature_selection import RFE, VarianceThreshold, SelectFromModel
from sklearn.feature_selection import SelectKBest, mutual_info_regression, mutual_info_classif, chi2
from sklearn import metrics
from sklearn.model_selection import cross_validate, train_test_split
from sklearn.preprocessing import scale

In [2]:
subset = pd.read_csv('C:/Users/YLI116/downloads/subset.csv')
subset

,EduLevelAdults(DMDEDUC2),HouseholdIncome(INDHHIN2),Protein(gm)DR1TPROT,Sugar(gm)DR1TSUGR,fiber(gm)DR1TFIBE,Cholesterol(mg)DR1TCHOL,VitaminC(mg)DR1TVC,Iron(mg)DR1TIRON,$SpentOnEatingOutPastMonth(CBD120),#MealsFastFoodOrPizzaPlace(DBD900),...,TypeOfWorkDoneLast weekOCD150,HoursWorkedLastWeekAtAllJobs(OCQ180),NumberOfMonthsWorkingInMainJob(OCD270),HoursWatchTVorVideosPast30Days(PAQ710),HoursUseComputerPast30Days(PAQ715),HowMuchSleepDoYouGet(hours)SLD010H,FirstSexAge(SXD031),AgeWhenHeaviestWeight(WHQ150),log_vitaminA,depression
0,3,4,43.63,176.47,10.8,209,241.4,8.41,0,8,...,4,35,20,2,8,7,15,62,4.70,1
1,3,7,338.13,44.99,16.7,2584,2.3,26.88,40,4,...,1,50,420,4,8,9,14,25,7.31,2
2,4,10,64.61,102.90,9.9,88,195.1,17.57,40,0,...,4,35,20,4,0,8,17,35,6.48,0
3,5,15,55.24,87.78,12.3,41,236.5,17.72,0,4,...,4,35,20,1,1,9,17,60,6.76,9
4,4,9,55.11,81.75,22.6,534,224.4,9.52,60,14,...,1,56,372,5,8,5,15,54,5.87,19
5,5,10,91.15,128.23,24.8,152,72.0,20.10,200,1,...,4,35,20,0,0,9,18,61,7.22,2
6,3,4,42.26,85.92,13.1,39,79.4,7.65,50,4,...,1,38,84,1,8,6,17,45,6.20,1
7,2,3,38.09,98.01,7.0,81,9.1,10.25,0,4,...,1,20,36,3,0,7,17,42,2.40,3
8,5,15,139.21,83.91,37.6,629,77.5,15.20,50,1,...,1,40,24,0,8,8,17,20,7.26,0
9,5,14,39.40,61.42,14.9,80,168.3,7.00,100,1,...,4,35,20,3,1,6,17,68,6.41,6


In [3]:
train = subset.iloc[:,:-1]
train.head()

,EduLevelAdults(DMDEDUC2),HouseholdIncome(INDHHIN2),Protein(gm)DR1TPROT,Sugar(gm)DR1TSUGR,fiber(gm)DR1TFIBE,Cholesterol(mg)DR1TCHOL,VitaminC(mg)DR1TVC,Iron(mg)DR1TIRON,$SpentOnEatingOutPastMonth(CBD120),#MealsFastFoodOrPizzaPlace(DBD900),MinutesOutdoors9-5NotWorkDay(DED125),TypeOfWorkDoneLast weekOCD150,HoursWorkedLastWeekAtAllJobs(OCQ180),NumberOfMonthsWorkingInMainJob(OCD270),HoursWatchTVorVideosPast30Days(PAQ710),HoursUseComputerPast30Days(PAQ715),HowMuchSleepDoYouGet(hours)SLD010H,FirstSexAge(SXD031),AgeWhenHeaviestWeight(WHQ150),log_vitaminA
0,3,4,43.63,176.47,10.8,209,241.4,8.41,0,8,120,4,35,20,2,8,7,15,62,4.70
1,3,7,338.13,44.99,16.7,2584,2.3,26.88,40,4,120,1,50,420,4,8,9,14,25,7.31
2,4,10,64.61,102.90,9.9,88,195.1,17.57,40,0,120,4,35,20,4,0,8,17,35,6.48
3,5,15,55.24,87.78,12.3,41,236.5,17.72,0,4,120,4,35,20,1,1,9,17,60,6.76
4,4,9,55.11,81.75,22.6,534,224.4,9.52,60,14,60,1,56,372,5,8,5,15,54,5.87


In [4]:
target = subset.depression

In [5]:
data_np = np.asarray(train)

In [6]:
data_np=scale(data_np)

In [7]:
target_np = np.asarray(target)

In [8]:
target_np=scale(target_np)

C:\Anaconda3\lib\site-packages\sklearn\utils\validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64 by the scale function.
  warnings.warn(msg, DataConversionWarning)


In [9]:
data_train, data_test, target_train, target_test = train_test_split(data_np, target_np, test_size=0.35)

In [10]:
scorers = {'Neg_MSE': 'neg_mean_squared_error', 'expl_var': 'explained_variance'} 

start_ts=time.time()
#rgr=SVR(C =1.0, gamma = 0.1, kernel = 'linear')
rgr = SVR(kernel='linear', C=100, gamma='auto')
scores= cross_validate(rgr, data_train, target_train, scoring=scorers, cv=5)
scores_RMSE = np.asarray([math.sqrt(-x) for x in scores['test_Neg_MSE']]) #Turns negative MSE scores into RMSE
scores_Expl_Var = scores['test_expl_var']
print("SVM RMSE:: %0.2f (+/- %0.2f)" % ((scores_RMSE.mean()), (scores_RMSE.std() * 2)))
print("SVM Expl Var: %0.2f (+/- %0.2f)" % ((scores_Expl_Var.mean()), (scores_Expl_Var.std() * 2)))
print("CV Runtime:", time.time()-start_ts)

SVM RMSE:: 0.99 (+/- 0.12)
SVM Expl Var: 0.08 (+/- 0.04)
CV Runtime: 369.275593996048


In [14]:
rbf_rgr = SVR(kernel='rbf', C=100, gamma='auto')
scores= cross_validate(rbf_rgr, data_train, target_train, scoring=scorers, cv=5)
scores_RMSE = np.asarray([math.sqrt(-x) for x in scores['test_Neg_MSE']]) #Turns negative MSE scores into RMSE
scores_Expl_Var = scores['test_expl_var']
print("SVM RMSE:: %0.2f (+/- %0.2f)" % ((scores_RMSE.mean()), (scores_RMSE.std() * 2)))
print("SVM Expl Var: %0.2f (+/- %0.2f)" % ((scores_Expl_Var.mean()), (scores_Expl_Var.std() * 2)))
print("CV Runtime:", time.time()-start_ts)

SVM RMSE:: 1.29 (+/- 0.08)
SVM Expl Var: -0.64 (+/- 0.18)
CV Runtime: 977.7625896930695


In [15]:
sigmoid_rgr = SVR(kernel='sigmoid', C=100, gamma='auto')
scores= cross_validate(sigmoid_rgr, data_train, target_train, scoring=scorers, cv=5)
scores_RMSE = np.asarray([math.sqrt(-x) for x in scores['test_Neg_MSE']]) #Turns negative MSE scores into RMSE
scores_Expl_Var = scores['test_expl_var']
print("SVM RMSE:: %0.2f (+/- %0.2f)" % ((scores_RMSE.mean()), (scores_RMSE.std() * 2)))
print("SVM Expl Var: %0.2f (+/- %0.2f)" % ((scores_Expl_Var.mean()), (scores_Expl_Var.std() * 2)))
print("CV Runtime:", time.time()-start_ts)

SVM RMSE:: 1529.50 (+/- 774.04)
SVM Expl Var: -2516605.61 (+/- 2491101.63)
CV Runtime: 1108.2519314289093


In [17]:
poly_rgr = SVR(kernel='poly', C=100, gamma='auto')
scores= cross_validate(poly_rgr, data_train, target_train, scoring=scorers, cv=5)
scores_RMSE = np.asarray([math.sqrt(-x) for x in scores['test_Neg_MSE']]) #Turns negative MSE scores into RMSE
scores_Expl_Var = scores['test_expl_var']
print("SVM RMSE:: %0.2f (+/- %0.2f)" % ((scores_RMSE.mean()), (scores_RMSE.std() * 2)))
print("SVM Expl Var: %0.2f (+/- %0.2f)" % ((scores_Expl_Var.mean()), (scores_Expl_Var.std() * 2)))
print("CV Runtime:", time.time()-start_ts)

SVM RMSE:: 2.24 (+/- 0.31)
SVM Expl Var: -4.04 (+/- 2.07)
CV Runtime: 1796.8627281188965


In [19]:
file1 = csv.reader(open('C:/Users/YLI116/downloads/subset.csv'), quotechar = '"')
header = next(file1)
#header_new

In [28]:
from sklearn.feature_selection import RFE, VarianceThreshold, SelectFromModel

In [29]:
# feature selection using svm 
rgr = SVR(kernel='linear', C=100, gamma='auto')
sel = SelectFromModel(rgr, prefit=False, threshold='mean')
fit_mod=sel.fit(data_np, target_np)  
sel_idx=fit_mod.get_support()


temp=[]

for i in range(1, len(header)):
    
    temp.append([header[i], float(scores[i-1])])

print('Ranked Features')
temp_sort=sorted(temp, key=itemgetter(1), reverse=True)
for i in range(len(temp_sort)):
    print(i, temp_sort[i][0], ':', temp_sort[i][1])
print('\n')




KeyError: 0

In [30]:
rgr = SVR(kernel='linear', C=100, gamma='auto')
sel = SelectFromModel(rgr, prefit=False, threshold='mean')
fit_mod=sel.fit(data_np, target_np)  
sel_idx=fit_mod.get_support()

temp=[]
temp_idx=[]
temp_del=[]
for i in range(len(data_np[0])):
    if sel_idx[i]==1:
        #Selected Features get added to temp header
        temp.append(header[i+1])
        temp_idx.append(i)
    else:                                                                       #Indexes of non-selected features get added to delete array
        temp_del.append(i)
print('Selected', temp)
print('Features (total/selected):', len(data_np[0]), len(temp))
print('\n')

Selected ['HouseholdIncome(INDHHIN2)', 'fiber(gm)DR1TFIBE', 'VitaminC(mg)DR1TVC', '$SpentOnEatingOutPastMonth(CBD120)', 'TypeOfWorkDoneLast weekOCD150', 'HoursWorkedLastWeekAtAllJobs(OCQ180)', 'FirstSexAge(SXD031)', 'AgeWhenHeaviestWeight(WHQ150)']
Features (total/selected): 20 8


